This notebook is based on the following sources: 

https://www.kaggle.com/code/parulpandey/a-guide-to-handling-missing-values-in-python (Parul Pandey)
https://www.kaggle.com/code/dansbecker/handling-missing-values/notebook (Dan S. Baker)
https://www.kaggle.com/code/alexisbcook/missing-values (Alexis B. Cook)
https://www.kaggle.com/code/rtatman/data-cleaning-challenge-handling-missing-values (Rachael Tatman)
https://www.kaggle.com/code/twinkle0705/a-comprehensive-guide-to-handle-missing-values (Twinkle Khanna)

We discuss several methods for handling missing values in datasets. We evaluate each method by training a machine learning model after curating the dataset, and compare their performance. 

Source: https://www.kaggle.com/code/parulpandey/a-guide-to-handling-missing-values-in-python

# Handling Missing Values in Python

![](https://imgur.com/68u0dD2.png)

>Real world data often contains a lot of missing values. The main reasons for incomplete data are

| Reason for missing Data |  
|--|
|Data doesn't exist  |  
|Data not collected due to human error.  |  
|Data deleted accidently  |  
|                         |

Many machine learning models need the datasets to be complete. We demonstrate in this notebook several methods to handle missing values in a dataset. There are also machine learning models, such as `XGBoost` and `LightGBM` that can handle incomplete datasets (find out in such cases how they do that).

## Loading necessary libraries and datasets

In [ ]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression

import missingno as msno

In [ ]:
# Reset the seed of the random number generator, for reproducibility purposes

import os

def reset_seed(SEED = 0):
    """Reset the seed for every random library in use (System, numpy)"""

    os.environ['PYTHONHASHSEED']=str(SEED)
    np.random.seed(SEED)


reset_seed(150)

In [ ]:
# A nice visualization of the missing values in a dataframe
# credit: https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction. 

def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
# A function to train a classifier and to report a metric on the validation data
# Used to evaluate the quality of different imputation methods
# After filling in the missing values, we apply this function to see how well the data can be learned

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

def score_datasetClassifier(X_train, X_valid, y_train, y_valid):
    model = RandomForestClassifier(
        n_estimators = 100, 
        criterion = 'gini',
        max_depth = 5,
        min_samples_split = 10,
        random_state = 180,
        verbose = 1,  
        n_jobs = -1,
    )
    model.fit(X_train, y_train)
    preds_valid = model.predict(X_valid)
    return roc_auc_score(y_valid, preds_valid)#, pos_label='1')

## Reading in the Titanic dataset
We use the dataset on the passengers on the Titanic: some of their personal data, cabine and ticketing data, and whether they survived or not. The data is highly incomplete, and the goal of this notebook is to test some methods to deal with such data. 

In [ ]:
from sklearn.datasets import fetch_openml

titanic_X, titanic_y = fetch_openml(
    data_id=40945,
    as_frame=True,
    return_X_y=True,
    parser = 'auto'
)

print('The Titanic dataset:')
print(titanic_X.info())

print('\n The labels:')
print(titanic_y.info())

In [ ]:
# We replace the category features in the dataset with numerical encodings.

titanic_X_category_columns = titanic_X.select_dtypes(['category']).columns
titanic_X[titanic_X_category_columns] = titanic_X[titanic_X_category_columns].apply(lambda x: x.cat.codes)

print(titanic_X.info())

In [ ]:
titanic_X.head()

In [ ]:
titanic_y.head()

In [ ]:
# Check the missing values in the Titanic dataset

titanic_X_missing= missing_values_table(titanic_X)
titanic_X_missing

In [ ]:
# Split the training dataset into train+validation and test

from sklearn.model_selection import train_test_split

(titanic_X_train_valid, titanic_X_test, 
 titanic_y_train_valid, titanic_y_test) = train_test_split(titanic_X,
                                                           titanic_y,
                                                           test_size=0.2,
                                                           random_state=2023,
                                                           stratify=titanic_y,
                                                           shuffle=True,
                                                          )

titanic_X_train_valid = titanic_X_train_valid.reset_index(drop=True)
titanic_X_test = titanic_X_test.reset_index(drop=True)
titanic_y_train_valid = titanic_y_train_valid.reset_index(drop=True)
titanic_y_test = titanic_y_test.reset_index(drop=True)



# Split the training dataset into training and validation

(titanic_X_train, titanic_X_valid, 
titanic_y_train, titanic_y_valid) = train_test_split(titanic_X_train_valid,
                                                     titanic_y_train_valid,
                                                     test_size=0.25,
                                                     random_state=2023,
                                                     stratify=titanic_y_train_valid,
                                                     shuffle=True,
                                                    )

del titanic_X_train_valid
del titanic_y_train_valid
del titanic_X
del titanic_y

titanic_X_train = titanic_X_train.reset_index(drop=True)
titanic_y_train = titanic_y_train.reset_index(drop=True)
titanic_X_valid = titanic_X_valid.reset_index(drop=True)
titanic_y_valid = titanic_y_valid.reset_index(drop=True)

# We merge the X and y to deal easier with the data processing in the two tables. 
#titanic_train = pd.merge(titanic_X_train, titanic_y_train, left_index=True, right_index=True)
#titanic_valid = pd.merge(titanic_X_valid, titanic_y_valid, left_index=True, right_index=True)
#titanic_test = pd.merge(titanic_X_test, titanic_y_test, left_index=True, right_index=True)

#del titanic_X_train
#del titanic_y_train
#del titanic_X_valid
#del titanic_y_valid
#del titanic_X_test
#del titanic_y_test

# Check the result of the data split

print('# of training samples:', len(titanic_X_train))
print(titanic_y_train.value_counts())

print('# of validation samples:', len(titanic_X_valid))
print(titanic_y_valid.value_counts())

print('# of test samples:', len(titanic_X_test))
print(titanic_y_test.value_counts())

## Examining the Target column 

>We create a model that predicts whether or not the passengers survived the sinking of the Titanic. Let's examine the distribution of these labels. 

In [ ]:
s = sns.countplot(x = titanic_y_train)
sizes=[]
for p in s.patches:
    height = p.get_height()
    sizes.append(height)
    s.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/len(titanic_y_train)*100),
            ha="center", fontsize=14) 

Here:
* 0: Did not Survive while 
* 1: Survived. 

# Detecting Missing values

## Detecting missing values numerically 

>The first step is to detect the count/percentage of missing values in every column of the dataset. This will give an idea about the distribution of missing values.

In [ ]:
titanic_X_train_missing= missing_values_table(titanic_X_train)
titanic_X_train_missing

## Detecting missing data visually using Missingno library

>To graphically analyse the missingness of the data, let's use a library called [Missingno](https://github.com/ResidentMario/missingno). It is a package for graphical analysis of missing values.

In [ ]:
msno.bar(titanic_X_train)

>The bar chart above gives a quick graphical overview of the completeness of the dataset. We can see that the age, fare, cabin, embarked, boat, body, and home_dest columns have missing values. 

### Visualizing the locations of the missing data 

>The [msno.matrix](https://github.com/ResidentMario/missingno#matrix) nullity matrix is a data-dense display which lets you quickly visually pick out patterns in data completion.


In [ ]:
msno.matrix(titanic_X_train)

>* The plot has a white horizontal line, for every cell with a missing values. For instance, in the 'embarked' column there is only one instance of missing data, hence one white lines.
>
>* The sparkline on the right gives an idea of the general shape of the completeness of the data and points out the row with the minimum nullities and the total number of columns in a given dataset, at the bottom.

# Reasons for Missing Values 

>Before we start treating the missing values ,it is important to understand the various reasons for the missingness in data. Broadly speaking, there can be three possible reasons:

Source: Rubin, D. B. (1976). Inference and Missing Data. Biometrika, 63(3), 581–592. https://doi.org/10.2307/2335739

1. Missing Completely at Random (MCAR)
>The missingness on a given variable (Y) are not associated with other variables in a given data set or with the variable (Y) itself. In other words, there is no particular reason for the missing values. 

2. Missing at Random (MAR)
>The missingness on a given variable (Y) is associated with other variables in the data set. For example, males may be less likely to answer to a question on whether they suffer from depression, regardless of the level of their depression.  

3. Missing Not at Random (MNAR)
>The missingness on a given variable (Y) depend on unobserved data or the value of the missing data itself. For example, people suffering from depression may be less likely to answer to a question on whether they suffer from depression, the higher their level of depression is. 

## Exploring the reasons for missing data using matrix plot 

In [ ]:
msno.matrix(titanic_X_train)

In [ ]:
#sorted by Cabin
sorted = titanic_X_train.sort_values('cabin')
msno.matrix(sorted)
del sorted

We seem to have slightly more data in columns 'Boat' and 'home_dest' when the data on 'cabin' is available. There may be a small correlation here: the passengers in first class may have been better documented and may have gotten places on boats to a larger extent. We can check this by sorting on 'pclass'. 

In [ ]:
#sorted by pclass
sorted = titanic_X_train.sort_values('pclass')
msno.matrix(sorted)
print(sorted['pclass'].value_counts())
del sorted

Our hypothesis seems reinforced by this plot: the passengers in 1st class seem to have better documented data about their cabin. The same for passengers in 1st and 2nd class on their home destination. 

We can also use a heatmap to check the correlations between the features when it comes to missing data. 

In [ ]:
msno.heatmap(titanic_X_train)

The heatmap function shows that there are some correlations between cabin and boat and home destination, as observed above. None of the correlations are very strong. Low correlations suggest that the data are MAR.

# Treating Missing values  


![](https://imgur.com/tBvdfyX.png)

### Pairwise Deletion 

>In Pairwise deletion, only the missing values are deleted. All operations in pandas like mean,sum etc intrinsically skip missing values.

In [ ]:
titanic_X_train.isnull().sum()


### Listwise Deletion/ Dropping rows

>During Listwise deletion, complete rows(which contain the missing values) are deleted. As a result, it is also called Complete Case deletion. 

Let's drop all rows where there is at least one missing value in any of the columns.

In [ ]:
mycopy = titanic_X_train.dropna(axis=0)
mycopy.info()
del mycopy

#### Oops: no data left to analyse!!! This is because every row in our dataset had at least one missing value. 


### Dropping complete columns 

We might have better luck removing all the columns that have at least one missing value instead.

In [ ]:
mycopy = titanic_X_train.dropna(axis=1)
mycopy.info()

In [ ]:
# How much data did we lose?

print('Columns in the original training set: ', titanic_X_train.shape[1])
print('Columns after dropping N/A: ', mycopy.shape[1])

del mycopy

This is quite a drastic loss. We should rather focus on dropping only the columns where the majority of data is missing. The other columns we deal with via imputations. 

In [ ]:
titanic_X_train.isna().sum()

The majority of data is missing from columns 'cabin', 'boat', 'body', and 'home.dest'. We drop them from the dataset.

In [ ]:
titanic_X_train.drop(['cabin', 'boat', 'body', 'home.dest'], axis=1, inplace=True)
titanic_X_valid.drop(['cabin', 'boat', 'body', 'home.dest'], axis=1, inplace=True)
titanic_X_test.drop(['cabin', 'boat', 'body', 'home.dest'], axis=1, inplace=True)

titanic_X_train.info()

We can imputate the missing values on 'age' (157) and 'fare' (1).

## Imputations Techniques

![](https://imgur.com/bL0iHde.png)

>Imputation refers to replacing missing data with substituted values. This can be done in several different ways, some of them discussed below. 

### Basic Imputation Techniques
  
  - Imputating with a constant value
  - Imputation using a statistical indicator (mean, median or most frequent) of each column in which the missing values are located

#### A simple imputation technique: replace N/A with a constant value, e.g., 0.

In [ ]:
# Replace N/A with a constant value

from sklearn.impute import SimpleImputer

mycopy_train = titanic_X_train.copy(deep=True)
print('Missing data before imputation:\n ', mycopy_train.isnull().sum())

constant_imputer = SimpleImputer(strategy='constant', fill_value=0) # imputing using constant value
constant_imputer.fit(mycopy_train)

mycopy_train = pd.DataFrame(
    constant_imputer.transform(mycopy_train),
    columns = titanic_X_train.columns)

#mycopy_train.iloc[:,:] = constant_imputer.transform(mycopy_train)
print('\n No missing data after imputation:\n', mycopy_train.isnull().sum())

We can evaluate the quality of this imputation technique through how well the new dataset can be learned with a Random Forest Regressor. We will use the same model architecture to evalauate each imputation technique. 

In [ ]:
# Apply the same transformation to the validation data

mycopy_valid = titanic_X_valid.copy(deep=True)
#mycopy_valid.iloc[:,:] = constant_imputer.transform(mycopy_valid)
mycopy_valid = pd.DataFrame(
    constant_imputer.fit_transform(mycopy_valid),
    columns = titanic_X_valid.columns)


# Train the RF model and get its score on the validation data

print(score_datasetClassifier(
    mycopy_train[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']], 
    mycopy_valid[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']], 
    titanic_y_train, 
    titanic_y_valid))

del mycopy_train
del mycopy_valid

# del constant_imputer

#### Another imputation technique: replace N/A with the most frquent value in that feature. 

In [ ]:
# Replace N/A with the most frequent value in that feature

from sklearn.impute import SimpleImputer

mycopy_train = titanic_X_train.copy(deep=True)
print('Missing data before imputation:\n', mycopy_train.isnull().sum())

frequent_imputer = SimpleImputer(strategy='most_frequent') # imputing using constant value
frequent_imputer.fit(mycopy_train)
mycopy_train = pd.DataFrame(
    frequent_imputer.transform(mycopy_train),
    columns = titanic_X_train.columns)

print('\n No missing data after imputation:\n', mycopy_train.isnull().sum())

We can evaluate the quality of this imputation technique through how well the new dataset can be learned with a Random Forest Regressor. We will use the same model architecture to evalauate each imputation technique. 

In [ ]:
# Apply the same transformation to the validation data

mycopy_valid = titanic_X_valid.copy(deep=True)
mycopy_valid = pd.DataFrame(
    frequent_imputer.transform(mycopy_valid),
    columns = titanic_X_valid.columns)

print(score_datasetClassifier(
    mycopy_train[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']], 
    mycopy_valid[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']], 
    titanic_y_train, 
    titanic_y_valid))

del mycopy_train
del mycopy_valid

# del frequent_imputer

### Advanced Imputation Techniques 

#### K-Nearest Neighbor Imputation
>Each missing feature is imputed using values from n_neighbors nearest neighbors that have a value for the feature. The feature of the neighbors are averaged uniformly or weighted by distance to each neighbor. 

In [ ]:
from sklearn.impute import KNNImputer
mycopy_train = titanic_X_train.copy(deep=True)

knn_imputer = KNNImputer(n_neighbors=7, weights="uniform")
knn_imputer.fit(mycopy_train[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']])

mycopy_train[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']] = pd.DataFrame(
    knn_imputer.transform(mycopy_train[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]),
    columns = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked'])

print('\n No missing data after imputation:\n', mycopy_train.isnull().sum())

We can evaluate the quality of this imputation technique through how well the new dataset can be learned with a Random Forest Regressor. We will use the same model architecture to evalauate each imputation technique. 

In [ ]:
# Apply the same transformation to the validation data

mycopy_valid = titanic_X_valid.copy(deep=True)
mycopy_valid[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']] = pd.DataFrame(
    knn_imputer.transform(mycopy_valid[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]),
    columns = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked'])

# Train the RF model and get its R2 score on the validation data
print(score_datasetClassifier(mycopy_train[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']], 
                    mycopy_valid[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']], 
                    titanic_y_train, 
                    titanic_y_valid))

del mycopy_train
del mycopy_valid

# del knn_imputer

#### Multivariate feature imputation - Multivariate imputation by chained equations (MICE)
A strategy for imputing missing values by modeling each feature with missing values as a function of other features in a round-robin fashion. It performns multiple regressions over random sample ofthe data, then takes the average ofthe multiple regression values and uses that value to impute the missing value. 

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
mycopy_train = titanic_X_train.copy(deep=True)

mice_imputer = IterativeImputer()
mice_imputer.fit(mycopy_train[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']])

mycopy_train[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']] = pd.DataFrame(
    mice_imputer.transform(mycopy_train[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]),
    columns = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked'])

print('\n No missing data after imputation:\n', mycopy_train.isnull().sum())

We can evaluate the quality of this imputation technique through how well the new dataset can be learned with a Random Forest Regressor. We will use the same model architecture to evalauate each imputation technique. 

In [ ]:
# Apply the same transformation to the validation data

mycopy_valid = titanic_X_valid.copy(deep=True)

mycopy_valid[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']] = pd.DataFrame(
    mice_imputer.transform(mycopy_valid[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]),
    columns = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked'])

# Train the RF model and get its R2 score on the validation data
print(score_datasetClassifier(mycopy_train[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']], 
                    mycopy_valid[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']], 
                    titanic_y_train, 
                    titanic_y_valid))

del mycopy_train
del mycopy_valid

# del mice_imputer

We conclude that all imputation techniques give roughly the same performance, with the knn- and mice-based imputation just slightly better. We check the result of the knn-based method on the test datatset.

In [ ]:
mycopy_train = titanic_X_train.copy(deep=True)
mycopy_train[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']] = pd.DataFrame(
    knn_imputer.transform(mycopy_train[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]),
    columns = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked'])

# Apply the same transformation to the test data

mycopy_test = titanic_X_test.copy(deep=True)
mycopy_test[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']] = pd.DataFrame(
    knn_imputer.transform(mycopy_test[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]),
    columns = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked'])

# Train the RF model and get its R2 score on the validation data
print(score_datasetClassifier(mycopy_train[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']], 
                    mycopy_test[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']], 
                    titanic_y_train, 
                    titanic_y_test))

del mycopy_train
del mycopy_test

# del knn_imputer

In [ ]:
del constant_imputer
del frequent_imputer
del knn_imputer
del mice_imputer

del titanic_X_train
del titanic_y_train
del titanic_X_valid
del titanic_y_valid
del titanic_X_test
del titanic_y_test

### Challenge

In this assignment you will load a dataset from OpenML and imputate the missing values with different methods. 

- Load the Ottawa Real Estate DataSet available at https://openml.org/search?type=data&status=active&id=43417&sort=runs. Consider the price as the labels of your dataset. 

>Q1: What is the data_id needed to load this dataset in your Python code?

>Q2: Which of the following features contain missing values: latitude	longitude	walkScore	propertyType	style	yearBuilt	bedrooms	bathrooms	parking	garage	lotDepth	lotFrontage	price? 

- The price in this dataset is written in US style, with commas separating every block of 3 digits. This will be problematic in your code. Convert the price from its string format to a numerical format. 
- Check the 'style' feature of your dataset.

>Q3 How many possible values does 'style' have in this dataset? 

>Q4 What is the 2nd largest class in 'style'?
- Consider for the rest of this assignment only the data in the largest class in 'style', i.e., "Detached". Also, drop the feature 'propertyType' from the dataset. 

>Q5 How many of the remaining features have missing values? 

>Q6 What is the feature missing the most values? 

- Split the data into train, validation, test with the exact same function available in this notebook.
- Use the 4 imputation methods demonstrated in this notebook to fill in the missing values. To evaluate the quality of the imputation methods, use the following function (in the same way as demonstrated in this notebook) on your train and validation datasets: 

def score_datasetRegressor(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(
        n_estimators = 100, 
        criterion = 'squared_error',
        max_depth = 10,
        min_samples_split = 20,
        min_samples_leaf = 5,
        random_state = 180,
        verbose = 1,  
        n_jobs = -1,
    )
    model.fit(X_train, y_train)
    preds_valid = model.predict(X_valid)
    return r2_score(y_valid, preds_valid)#, pos_label='1')
    
>Q7 Which imputation method had the best score on the validation dataset? 

>Q8 What was the score of the best method on the test dataset? Report the result using 2 decimals only. 